In [1]:
import numpy as np
import os
import time
from netCDF4 import Dataset
from numba import njit

#@njit
def gen_img():
    ny, nx = 1000, 1000
    img = np.random.random((ny, nx))
    return img

#@njit
def smooth_img(img):    
    new_img = np.zeros(img.shape)
    ny, nx = img.shape
    for j in range(1, ny-1):
        for i in range(1, nx-1):
            new_img[j, i] = (img[j-1, i-1] + img[j-1, i] + img[j-1, i+1] +
                             img[j,   i-1] + img[j,   i] + img[j,   i+1] +
                             img[j+1, i-1] + img[j+1, i] + img[j+1, i+1]) / 9

    return new_img
    
def write_img(img, job_id):
    pass


In [17]:
%%time 

for job_id in range(5):
    print('job', job_id)
    img = gen_img()
    img = smooth_img(img)
    write_img(img, job_id)

job 0
job 1
job 2
job 3
job 4
CPU times: user 43.8 s, sys: 178 ms, total: 44 s
Wall time: 44.3 s


In [18]:
img = gen_img()

In [23]:
%%time
img1 = smooth_img(img)

CPU times: user 9.2 s, sys: 78.7 ms, total: 9.28 s
Wall time: 9.26 s


In [22]:
%%time
img1 = njit(smooth_img)(img)

CPU times: user 379 ms, sys: 22.9 ms, total: 401 ms
Wall time: 382 ms


In [2]:
from dask.distributed import Client, LocalCluster, fire_and_forget
cluster = LocalCluster(n_workers=4, threads_per_worker=2)
client = Client(cluster)

In [6]:
cluster.scale(4)

In [3]:
for job_id in range(200):
    img = client.submit(gen_img, pure=False)
    img = client.submit(smooth_img, img)
    img = client.submit(np.fft.fft2, img)
    future = client.submit(write_img, img, job_id)
    fire_and_forget(future)
